In [59]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import spacy
import string
from unidecode import unidecode
nltk.download("stopwords")


pd.set_option('display.max_colwidth', 200)

#carregando arquivo
df = pd.read_excel(r"D:\Documents\ESTUDOS\POS GRADUACAO\TCC\base_nps_completa_original.xlsx")

#limpando pesquisas em outro idioma
df = df[df['surveyId'] == 'NPS Zendesk Português']
#removendo números
df['comentario_NPS'] = df['comentario_NPS'].apply(lambda x: re.sub(r'\d', '', str(x)))
#removendo "\n" dos comentários
df['comentario_NPS'].replace("\n", '', inplace = True)
#trocando comentários em branco ou sem letras por NONE
df['comentario_NPS'] = df['comentario_NPS'].apply(lambda x: x if re.fullmatch(r".*[a-zA-Z].*", x) != None else None)
df['comentario_NPS'].replace({'', 'NULL','nan'}, None, inplace = True)
#filtrando apenas pesquisas com comentários
df = df[df['comentario_NPS'].isnull() == False]
#transformando tudo em minúsculas
df['comentario_NPS'] = df['comentario_NPS'].apply(lambda x: x.lower())
#filtrando apenas as colunas necessárias
df = df[['metadata_ticket', 'resposta_NPS', 'rqt_tipocliente', 'comentario_NPS']]

#separando dataframes por tipo de ticket
df_internos = df[df['rqt_tipocliente'] == 'INTERNO']
df_externos = df[df['rqt_tipocliente'] != 'INTERNO']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Renan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
#funcao pra remocao de pontos, acentos e stopwords
def limpatokens(comentarios):
    tks_saida = []
    for comentario in comentarios:
        tokens = nltk.word_tokenize(comentario, language = 'portuguese')
        ptbr_stopwords = stopwords.words('portuguese')
        tokens = [t for t in tokens if t not in ptbr_stopwords]
        tokens = [t for t in tokens if t not in string.punctuation]
        tokens = [unidecode(t) for t in tokens]
        tks_saida.append(tokens)
    return tks_saida

In [64]:
#criando tokens limpos
lista_tokens = limpatokens(df['comentario_NPS']) 